# Distributed Machine Learning Project

## Loading Libraries

In [1]:
# Import Pyspark

import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Spark\\spark-3.2.0-bin-hadoop3.2'

In [2]:
# Loading Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark_Basics").getOrCreate()
from pyspark.sql.functions import approxCountDistinct, countDistinct, count, when, isnan, col, isnull


## Loading Dataset

In [4]:
# Uploading as Pyspark File

file = "OneDrive - Mississippi State University/FA2021/ML_Project_FA2021/Project Data/vehicles/vehicles.csv"

file_location = file
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type)\
.option("InferSchema", infer_schema)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load(file_location)

df.printSchema()

AnalysisException: Path does not exist: file:/C:/Users/dhful/OneDrive - Mississippi State University/FA2021/ML_Project_FA2021/Project Code/OneDrive - Mississippi State University/FA2021/ML_Project_FA2021/Project Data/vehicles/vehicles.csv

## Data Analysis

In [5]:
# Performing basic statistics using Pandas

Dataframe = pd.read_csv(file)

Dataframe.describe()

FileNotFoundError: [Errno 2] No such file or directory: 'OneDrive - Mississippi State University/FA2021/ML_Project_FA2021/Project Data/vehicles/vehicles.csv'

In [ ]:
# Cardinality of each Variable

def cardinality_calculation(df, cut_off=1):
    cardinality = df.select(*[approxCountDistinct(c).alias(c) for c in df.columns])
    
    ## convert to pandas for efficient calculations
    final_cardinality_df = cardinality.toPandas().transpose()
    final_cardinality_df.reset_index(inplace=True) 
    final_cardinality_df.rename(columns={0:'Cardinality'}, inplace=True) 
    
    vars_selected = final_cardinality_df['index'][final_cardinality_df['Cardinality'] <= cut_off] 
    
    return final_cardinality_df, vars_selected

cardinality_df, cardinality_vars_selected = cardinality_calculation(df)
cardinality_df

# We will lose zero features due to low cardinality

In [ ]:
# Missing Percentage


def missing_calculation(df, miss_percentage=0.45):
    
    missing = df.select(*[count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
    length_df = df.count()

    final_missing_df = missing.toPandas().transpose()
    final_missing_df.reset_index(inplace=True) 
    final_missing_df.rename(columns={0:'missing_count'}, inplace=True) 
    final_missing_df['missing_percentage'] = final_missing_df['missing_count']/length_df
    
    vars_selected = final_missing_df['index'][final_missing_df['missing_percentage'] >= miss_percentage] 
    
    return final_missing_df, vars_selected

missing_df, missing_vars_selected = missing_calculation(df)
missing_df


# As seen below there are several features with above acceptable missing percentage. That means we need to 
# - remove those datapoints.

## Feature Engineering

In [ ]:
#Converting to Pandas 

df2 = df.toPandas()
missing_df_pd = spark.createDataFrame(missing_df)
missing_df2 = missing_df_pd.toPandas()



In [ ]:
# Deleting Columns with too much percentage missing


th = .10
# Defining threshold for cutoff percentage

bad_column = []
for i,r in missing_df2.iterrows():
        c = r["index"]
        mp = r["missing_percentage"]
        if mp > th:
            bad_column.append(c)

for i in bad_column:
    for j in df2.columns:
        if j == i:
            df2 = df2.drop(j, axis = 1)
            
print(len(df2.columns))
print(df2.columns)


In [ ]:
# Changing to String values to Numeric values

Sdf2 = spark.dataframe(df2)

for i in columns:
    
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_Indexed")
indexed = indexer.fit(df).transform(df)

In [ ]:
spark.stop()